In [1]:
import sys
!git clone https://github.com/HumanMLLM/LLaVA-Scissor.git
!ls -la

Cloning into 'LLaVA-Scissor'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 237 (delta 70), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (237/237), 1.95 MiB | 6.23 MiB/s, done.
Resolving deltas: 100% (70/70), done.
total 20
drwxr-xr-x 1 root root 4096 Oct 19 13:32 .
drwxr-xr-x 1 root root 4096 Oct 19 13:30 ..
drwxr-xr-x 4 root root 4096 Oct 16 13:41 .config
drwxr-xr-x 7 root root 4096 Oct 19 13:32 LLaVA-Scissor
drwxr-xr-x 1 root root 4096 Oct 16 13:41 sample_data


In [2]:
sys.path.append('./LLaVA-Scissor')

In [3]:
# Install decord and other video processing libraries
!pip install decord
!pip install av
!pip install pyav
!pip install opencv-python
!pip install moviepy

# Also install the core dependencies
!pip install "accelerate==0.28.0"
!pip install matplotlib huggingface_hub
!pip install "transformers==4.37.2"
!pip install "torch==2.1.2" "torchvision==0.16.2"
!pip install "pillow>=9.0.0"
!pip install "numpy>=1.21.0"
!pip install "open-clip-torch>=2.20.0"
!pip install "datasets==2.16.1"

# Install the package in development mode
!cd /content/LLaVA-Scissor && pip install -e .[train]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 17.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pyav (from versions: none)
ERROR: No matching distribution found for pyav
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.10.1
    Uninstalling accelerate-1.10.1:
      Successfully uninstalled accelerate-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 127.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found

In [4]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from llava.conversation import conv_templates, SeparatorStyle
from huggingface_hub import snapshot_download, hf_hub_download
import bisect
import shutil
import numpy as np
from PIL import Image
import requests
import copy
import warnings
from decord import VideoReader, cpu
from datasets import load_dataset

/content/LLaVA-Scissor/llava/model/llava_arch.py:217: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
  if slower_img_feat is not 0:


from .language_model.llava_llama import LlavaLlamaForCausalLM, LlavaConfig
from .language_model.llava_qwen import LlavaQwenForCausalLM, LlavaQwenConfig
from .language_model.llava_mistral import LlavaMistralForCausalLM, LlavaMistralConfig
from .language_model.llava_mixtral import LlavaMixtralForCausalLM, LlavaMixtralConfig
from .language_model.llava_qwen_zip import LlavaQwenZipForCausalLM, LlavaQwenZipConfig


/content/LLaVA-Scissor/llava/model/llava_arch_zip.py:320: SyntaxWarning: "is not" with 'int' literal. Did you mean "!="?
  if slower_img_feat is not 0:
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
data_list = {
    "Action Sequence": ("action_sequence.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Action Prediction": ("action_prediction.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Action Antonym": ("action_antonym.json", "ssv2_video/", "video", False),
    "Fine-grained Action": ("fine_grained_action.json", "Moments_in_Time_Raw/videos/", "video", False),
    "Unexpected Action": ("unexpected_action.json", "FunQA_test/test/", "video", False),
    "Object Existence": ("object_existence.json", "clevrer/video_validation/", "video", False),
    "Object Interaction": ("object_interaction.json", "star/Charades_v1_480/", "video", True), # has start & end
    "Object Shuffle": ("object_shuffle.json", "perception/videos/", "video", False),
    "Moving Direction": ("moving_direction.json", "clevrer/video_validation/", "video", False),
    "Action Localization": ("action_localization.json", "sta/sta_video/", "video", True),  # has start & end
    "Scene Transition": ("scene_transition.json", "scene_qa/video/", "video", False),
    "Action Count": ("action_count.json", "perception/videos/", "video", False),
    "Moving Count": ("moving_count.json", "clevrer/video_validation/", "video", False),
    "Moving Attribute": ("moving_attribute.json", "clevrer/video_validation/", "video", False),
    "State Change": ("state_change.json", "perception/videos/", "video", False),
    "Fine-grained Pose": ("fine_grained_pose.json", "nturgbd/", "video", False),
    "Character Order": ("character_order.json", "perception/videos/", "video", False),
    "Egocentric Navigation": ("egocentric_navigation.json", "vlnqa/", "video", False),
    "Episodic Reasoning": ("episodic_reasoning.json", "tvqa/frames_fps3_hq/", "frame", True),  # has start & end, read frame
    "Counterfactual Inference": ("counterfactual_inference.json", "clevrer/video_validation/", "video", False),
}

data_dir = "dataset"
if not os.path.exists(data_dir):
    os.mkdir("dataset")

def read_video_pyav(video_path, start, end, n_frames=8):
    """
    Reads a video for given start-end timestamps interval
    and uniformly samples 8 frames of it
    """
    container = av.open(video_path)
    video = container.streams.get(0)[0]

    av_timestamps = [
        int(packet.pts * video.time_base) for packet in container.demux(video) if packet.pts is not None
    ]

    av_timestamps.sort()
    start_id = bisect.bisect_left(av_timestamps, start)
    end_id = bisect.bisect_left(av_timestamps, end)

    # in case it is a very short video, lets take a longer duration and sample
    if end_id  - start_id < 10:
        end_id += 10
        start_id -= 10

    end_id = min(len(av_timestamps) - 1, end_id)
    start_id = max(1, start_id)

    # We sample n_frames frames for tuning following the original paper
    # But we can increase the number of frames for longer videos and check out if it helps performance
    # Change the below "n_frames" to any number of frames you want, and note that more frames -> more computational resources needed
    indices = np.linspace(start_id, end_id, n_frames).astype(int)

    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_id:
            break
        if i >= start_id and i in indices:
            frames.append(frame)
    assert len(frames) == n_frames, f"Got {len(frames)} frames but should be {n_frames}. Check the indices: {indices};, start_id: {start_id}, end_id: {end_id}. Len of video is {len(av_timestamps)} frames."
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def collate_read_video(example, path):
    # Some datasets have a start-end interval, so we try to get it if exists.
    # Otherwise just set a very large end timestamp
    clip = read_video_pyav(f'{path}/{example["video"]}', example.get("start", 1), example.get("end", 1e+10))
    example["clip"] = clip
    return example

TASK_NAME = "Moving Count"
annotation_fn, video_dir, video_type, has_clip = data_list[TASK_NAME]

annotation_fn_local = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='json/' + annotation_fn,
    repo_type="dataset",
    local_dir=data_dir
)

video_zip_name = video_dir.split("/")[0] + ".zip"
videos_zip = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='video/' + video_zip_name,
    repo_type="dataset",
    local_dir=data_dir
)

for zip_file in os.listdir(f"{data_dir}/video"):
    if zip_file.endswith(".zip"):
        shutil.unpack_archive(
            f"{data_dir}/video/{zip_file}",
            f"{data_dir}/video/videos_unzipped/"
        )

ds = load_dataset("json", data_files=annotation_fn_local, split="train")
ds

has_missing = False
for sample in ds:
    if not os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{sample['video']}"):
        print(f"Video `{sample['video']}` does not exists!")
        has_missing = True

print(f"Dataset length = {len(ds)}")
if has_missing:
    ds = ds.filter(lambda x: os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{x['video']}"))

print(f"Dataset length = {len(ds)}")

moving_count.json: 0.00B [00:00, ?B/s]

video/clevrer.zip:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset length = 200
Dataset length = 200


In [6]:
ds

Dataset({
    features: ['answer', 'question', 'video', 'candidates'],
    num_rows: 200
})

In [7]:
from pathlib import Path

unzipped_dir = f"{data_dir}/video/videos_unzipped"
video_subdir = Path(video_dir).name
video_path = os.path.join(unzipped_dir, "star", video_subdir)

In [8]:
video_filename = ds[0]['video']
full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"

In [9]:
prompt = ds[0]['question']

In [10]:
!mkdir model_zoo
!cd model_zoo
'''
# (Optional) Using huggingface mirrors
!export HF_ENDPOINT=https://hf-mirror.com

# download the SIGLIP model form huggingface
!huggingface-cli download --resume-download google/siglip-so400m-patch14-384 --local-dir google/siglip-so400m-patch14-384

# download enhanced baseline model from huggingface
# 7B model
#!huggingface-cli download --resume-download BBBBCHAN/LLaVA-Scissor-baseline-7B --local-dir LLaVA-Scissor-baseline-7B
# 0.5B model
!huggingface-cli download --resume-download LLaVA-Scissor-baseline-0.5B --local-dir LLaVA-Scissor-baseline-0.5B

# We also support the original LLaVA-OneVision models.
#!huggingface-cli download --resume-download LLaVA-Scissor-baseline-7B --local-dir LLaVA-Scissor-baseline-7B
!huggingface-cli download --resume-download LLaVA-Scissor-baseline-0.5B --local-dir LLaVA-Scissor-baseline-0.5B
'''

'\n# (Optional) Using huggingface mirrors\n!export HF_ENDPOINT=https://hf-mirror.com\n\n# download the SIGLIP model form huggingface\n!huggingface-cli download --resume-download google/siglip-so400m-patch14-384 --local-dir google/siglip-so400m-patch14-384\n\n# download enhanced baseline model from huggingface\n# 7B model\n#!huggingface-cli download --resume-download BBBBCHAN/LLaVA-Scissor-baseline-7B --local-dir LLaVA-Scissor-baseline-7B\n# 0.5B model\n!huggingface-cli download --resume-download LLaVA-Scissor-baseline-0.5B --local-dir LLaVA-Scissor-baseline-0.5B\n\n# We also support the original LLaVA-OneVision models.\n#!huggingface-cli download --resume-download LLaVA-Scissor-baseline-7B --local-dir LLaVA-Scissor-baseline-7B\n!huggingface-cli download --resume-download LLaVA-Scissor-baseline-0.5B --local-dir LLaVA-Scissor-baseline-0.5B\n'

In [11]:
import os

# Check what's actually in model_zoo
print("Current directory:", os.getcwd())
print("Contents of model_zoo:")
!ls -la model_zoo/

# Check if we have any LLaVA models
if os.path.exists('model_zoo'):
    print("\nSearching for model directories:")
    for item in os.listdir('model_zoo'):
        item_path = os.path.join('model_zoo', item)
        if os.path.isdir(item_path):
            print(f"📁 {item}")
            # Check if it has model files
            model_files = [f for f in os.listdir(item_path) if f.endswith(('.bin', '.safetensors', 'config.json'))]
            if model_files:
                print(f"   Model files: {model_files[:3]}...")  # Show first 3

Current directory: /content
Contents of model_zoo:
total 8
drwxr-xr-x 2 root root 4096 Oct 19 13:35 .
drwxr-xr-x 1 root root 4096 Oct 19 13:35 ..

Searching for model directories:


In [12]:
import os

# Create the correct directory structure
os.makedirs('model_zoo/google', exist_ok=True)

# Download SIGLIP to the correct location
!hf download google/siglip-so400m-patch14-384 --local-dir model_zoo/google/siglip-so400m-patch14-384

print("SIGLIP model downloaded successfully!")

Fetching 9 files:   0% 0/9 [00:00<?, ?it/s]Downloading 'spiece.model' to 'model_zoo/google/siglip-so400m-patch14-384/.cache/huggingface/download/vj8E1loknrCNPSP8nWJC234Bff4=.1e5036bed065526c3c212dfbe288752391797c4bb1a284aa18c9a0b23fcaf8ec.incomplete'

tokenizer.json: 0.00B [00:00, ?B/s]Downloading 'preprocessor_config.json' to 'model_zoo/google/siglip-so400m-patch14-384/.cache/huggingface/download/PYH5dHjks7Ei0Yd3X0Z8xIwsCNQ=.051261b5a784a2e7e5f93de1f86d170d69a9efd9.incomplete'
tokenizer.json: 2.40MB [00:00, 68.7MB/s]
Download complete. Moving file to model_zoo/google/siglip-so400m-patch14-384/tokenizer.json

preprocessor_config.json: 100% 368/368 [00:00<00:00, 3.94MB/s]
Download complete. Moving file to model_zoo/google/siglip-so400m-patch14-384/preprocessor_config.json

config.json: 100% 576/576 [00:00<00:00, 6.07MB/s]
Download complete. Moving file to model_zoo/google/siglip-so400m-patch14-384/config.json

spiece.model:   0% 0.00/798k [00:00<?, ?B/s]Downloading 'README.md' to 'model

In [13]:
vision_tower_path = "model_zoo/google/siglip-so400m-patch14-384"
print(f"Vision tower path: {vision_tower_path}")
print(f"Path exists: {os.path.exists(vision_tower_path)}")

if os.path.exists(vision_tower_path):
    print("Contents of SIGLIP directory:")
    !ls -la {vision_tower_path}
else:
    print("❌ SIGLIP directory not found!")

Vision tower path: model_zoo/google/siglip-so400m-patch14-384
Path exists: True
Contents of SIGLIP directory:
total 3432808
drwxr-xr-x 3 root root       4096 Oct 19 13:35 .
drwxr-xr-x 3 root root       4096 Oct 19 13:35 ..
drwxr-xr-x 3 root root       4096 Oct 19 13:35 .cache
-rw-r--r-- 1 root root        576 Oct 19 13:35 config.json
-rw-r--r-- 1 root root       1519 Oct 19 13:35 .gitattributes
-rw-r--r-- 1 root root 3511950624 Oct 19 13:35 model.safetensors
-rw-r--r-- 1 root root        368 Oct 19 13:35 preprocessor_config.json
-rw-r--r-- 1 root root       4321 Oct 19 13:35 README.md
-rw-r--r-- 1 root root        409 Oct 19 13:35 special_tokens_map.json
-rw-r--r-- 1 root root     798330 Oct 19 13:35 spiece.model
-rw-r--r-- 1 root root        711 Oct 19 13:35 tokenizer_config.json
-rw-r--r-- 1 root root    2399357 Oct 19 13:35 tokenizer.json


In [24]:
import sys
sys.path.append('/content/LLaVA-Scissor')

warnings.filterwarnings("ignore")
# Load model
pretrained = "BBBBCHAN/LLaVA-Scissor-baseline-0.5B"
model_name = "llava_qwen"
device = "cuda"
device_map = "auto"
tokenizer, model, image_processor, max_length = load_pretrained_model(pretrained, None, model_name, device_map=device_map, attn_implementation="sdpa")

model.eval()

def load_video(video_path, max_frames_num):
    if type(video_path) == str:
        vr = VideoReader(video_path, ctx=cpu(0))
    else:
        vr = VideoReader(video_path[0], ctx=cpu(0))
    total_frame_num = len(vr)
    uniform_sampled_frames = np.linspace(0, total_frame_num - 1, max_frames_num, dtype=int)
    frame_idx = uniform_sampled_frames.tolist()
    spare_frames = vr.get_batch(frame_idx).asnumpy()
    return spare_frames  # (frames, height, width, channels)


# Load and process video
video_path = full_video_path
video_frames = load_video(video_path, 16)
print(video_frames.shape)
image_tensors = []
frames = image_processor.preprocess(video_frames, return_tensors="pt")["pixel_values"].half().cuda()
image_tensors.append(frames)

# Prepare conversation input
conv_template = "qwen_2"
question = f"{DEFAULT_IMAGE_TOKEN}\n{prompt}\nAnswer with only a number."
#question = f"{DEFAULT_IMAGE_TOKEN}\n{prompt}\nAnswer options: {ds[0]['candidates'][0]}, {ds[0]['candidates'][1]}, {ds[0]['candidates'][2]}, {ds[0]['candidates'][3]}."
#question = f"{DEFAULT_IMAGE_TOKEN}\n{prompt}."
conv = copy.deepcopy(conv_templates[conv_template])
conv.append_message(conv.roles[0], question)
conv.append_message(conv.roles[1], None)
prompt_question = conv.get_prompt()

input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
image_sizes = [frame.size for frame in video_frames]

# Generate response
cont = model.generate(
    input_ids,
    images=image_tensors,
    image_sizes=image_sizes,
    do_sample=False,
    temperature=0,
    max_new_tokens=4096,
    modalities=["video"],
)
text_outputs = tokenizer.batch_decode(cont, skip_special_tokens=True)
print(text_outputs[0])
print(text_outputs[0] == ds[0]['answer'])

Loaded LLaVA model: BBBBCHAN/LLaVA-Scissor-baseline-0.5B


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


llava_qwen


You are using a model of type qwen2 to instantiate a model of type llava_qwen. This is not supported for all configurations of models and can yield errors.


LlavaQwenConfig {
  "_name_or_path": "BBBBCHAN/LLaVA-Scissor-baseline-0.5B",
  "add_faster_video": false,
  "add_time_instruction": false,
  "architectures": [
    "LlavaQwenZipForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "faster_token_stride": 10,
  "force_sample": false,
  "hidden_act": "silu",
  "hidden_size": 896,
  "image_aspect_ratio": "anyres_max_9",
  "image_crop_resolution": null,
  "image_grid_pinpoints": [
    [
      384,
      384
    ],
    [
      384,
      768
    ],
    [
      384,
      1152
    ],
    [
      384,
      1536
    ],
    [
      384,
      1920
    ],
    [
      384,
      2304
    ],
    [
      768,
      384
    ],
    [
      768,
      768
    ],
    [
      768,
      1152
    ],
    [
      768,
      1536
    ],
    [
      768,
      1920
    ],
    [
      768,
      2304
    ],
    [
      1152,
      384
    ],
    [
      1152,
      768
    ],
    [
      1152,
      1152
    ],
  

In [25]:
import time
from tqdm import tqdm

comp_time = 0
correct_guessed = 0

model.eval()
with torch.no_grad():

    for i in tqdm(range(len(ds)), desc="Processing samples"):
        sample = ds[i]
        video_filename = sample['video']
        full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"
        prompt = sample['question']
        real_answer = sample['answer']
        video_path = full_video_path
        video_frames = load_video(video_path, 16)
        image_tensors = []
        frames = image_processor.preprocess(video_frames, return_tensors="pt")["pixel_values"].half().cuda()
        image_tensors.append(frames)
        conv_template = "qwen_2"
        question = f"{DEFAULT_IMAGE_TOKEN}\n{prompt}\nAnswer with only a number."
        conv = copy.deepcopy(conv_templates[conv_template])
        conv.append_message(conv.roles[0], question)
        conv.append_message(conv.roles[1], None)
        prompt_question = conv.get_prompt()
        input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
        image_sizes = [frame.size for frame in video_frames]

        start_time = time.time()

        cont = model.generate(
            input_ids,
            images=image_tensors,
            image_sizes=image_sizes,
            do_sample=False,
            temperature=0,
            max_new_tokens=4096,
            modalities=["video"],
        )
        text_outputs = tokenizer.batch_decode(cont, skip_special_tokens=True)

        end_time = time.time()
        elapsed_time = end_time - start_time
        comp_time += elapsed_time

        if text_outputs[0] == real_answer:
            correct_guessed +=1

accuracy = correct_guessed / len(ds)
print(f'Computational Time: {comp_time}\nAccuracy: {accuracy}')

Processing samples: 100%|██████████| 200/200 [06:44<00:00,  2.02s/it]

Computational Time: 326.3251643180847
Accuracy: 0.145


In [37]:
TASK_NAME = "Action Sequence"
annotation_fn, video_dir, video_type, has_clip = data_list[TASK_NAME]

annotation_fn_local = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='json/' + annotation_fn,
    repo_type="dataset",
    local_dir=data_dir
)

video_zip_name = video_dir.split("/")[0] + ".zip"
videos_zip = hf_hub_download(
    repo_id="OpenGVLab/MVBench",
    filename='video/' + video_zip_name,
    repo_type="dataset",
    local_dir=data_dir
)

for zip_file in os.listdir(f"{data_dir}/video"):
    if zip_file.endswith(".zip"):
        shutil.unpack_archive(
            f"{data_dir}/video/{zip_file}",
            f"{data_dir}/video/videos_unzipped/"
        )

ds = load_dataset("json", data_files=annotation_fn_local, split="train")
ds

has_missing = False
for sample in ds:
    if not os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{sample['video']}"):
        print(f"Video `{sample['video']}` does not exists!")
        has_missing = True

print(f"Dataset length = {len(ds)}")
if has_missing:
    ds = ds.filter(lambda x: os.path.exists(f"{data_dir}/video/videos_unzipped/{video_dir}/{x['video']}"))

print(f"Dataset length = {len(ds)}")

from pathlib import Path

unzipped_dir = f"{data_dir}/video/videos_unzipped"
video_subdir = Path(video_dir).name
video_path = os.path.join(unzipped_dir, "star", video_subdir)

video_filename = ds[1]['video']
full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"

prompt = ds[1]['question']

Video `EDXBD.mp4` does not exists!
Video `K47J5.mp4` does not exists!
Video `9MNZ5.mp4` does not exists!
Video `QXT9W.mp4` does not exists!
Video `ABHC6.mp4` does not exists!
Video `ALXUC.mp4` does not exists!
Video `BAUQE.mp4` does not exists!
Video `PHH6B.mp4` does not exists!
Video `MNC10.mp4` does not exists!
Video `W7CR5.mp4` does not exists!
Video `Q8UJ8.mp4` does not exists!
Video `X9WTR.mp4` does not exists!
Dataset length = 200
Dataset length = 188


In [38]:
comp_time = 0
correct_guessed = 0

model.eval()
with torch.no_grad():

    for i in tqdm(range(len(ds)), desc="Processing samples"):
        sample = ds[i]
        video_filename = sample['video']
        full_video_path = f"{unzipped_dir}/{video_dir}/{video_filename}"
        prompt = sample['question']
        real_answer = sample['answer']
        video_path = full_video_path
        video_frames = load_video(video_path, 16)
        image_tensors = []
        frames = image_processor.preprocess(video_frames, return_tensors="pt")["pixel_values"].half().cuda()
        image_tensors.append(frames)
        conv_template = "qwen_2"
        question = f"{DEFAULT_IMAGE_TOKEN}\n{prompt}\nChoose the correct answer:.\n{sample['candidates'][0]}, {sample['candidates'][1]}, {sample['candidates'][2]}, {sample['candidates'][3]}"
        conv = copy.deepcopy(conv_templates[conv_template])
        conv.append_message(conv.roles[0], question)
        conv.append_message(conv.roles[1], None)
        prompt_question = conv.get_prompt()
        input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
        image_sizes = [frame.size for frame in video_frames]

        start_time = time.time()

        cont = model.generate(
            input_ids,
            images=image_tensors,
            image_sizes=image_sizes,
            do_sample=False,
            temperature=0,
            max_new_tokens=4096,
            modalities=["video"],
        )
        text_outputs = tokenizer.batch_decode(cont, skip_special_tokens=True)

        end_time = time.time()
        elapsed_time = end_time - start_time
        comp_time += elapsed_time

        if text_outputs[0] == real_answer:
            correct_guessed +=1

accuracy = correct_guessed / len(ds)
print(f'Computational Time: {comp_time}\nAccuracy: {accuracy}')

Processing samples: 100%|██████████| 188/188 [07:28<00:00,  2.38s/it]

Computational Time: 324.522164106369
Accuracy: 0.22340425531914893
